In [2]:
from dolfinx import fem, io, geometry
from dolfinx.mesh import (locate_entities, create_rectangle, CellType, meshtags)
import dolfinx.cpp as _cpp
from ufl import Measure, pi, conditional, lt, gt,dx
from mpi4py import MPI
from matplotlib import cm, pyplot as plt
import numpy as np
import time

# Identificando o dóminio 

In [3]:
lx, ly = [2.0, 1.0]; Nx, Ny = [10, 5]
msh = create_rectangle(comm=MPI.COMM_WORLD,
            points=((0.0, 0.0), (lx,ly)), n=(Nx, Ny),
            cell_type=CellType.triangle, diagonal = _cpp.mesh.DiagonalType.crossed)

1) A partir da geometria do domínio

In [4]:
def tag_subdomains(msh, subdomains):
    # Identifies and marks subdomains accoring to locator function
    cell_indices, cell_markers = [], [] #List for facet indices and respective markers
    cdim = msh.topology.dim
    for (marker, locator) in subdomains:
        cells = locate_entities(msh, cdim, locator)
        cell_indices.append(cells)
        cell_markers.append(np.full_like(cells, marker))
    cell_indices = np.hstack(cell_indices).astype(np.int32)
    cell_markers = np.hstack(cell_markers).astype(np.int32)
    sorted_cells = np.argsort(cell_indices)
    cell_tag = meshtags(msh, cdim, cell_indices[sorted_cells], cell_markers[sorted_cells]) 
    return cell_tag


radius = 0.4; tol = 1e-8

def phi_expression(x):
    return  - np.sqrt((x[0] - 1.0)**2 + (x[1]-0.5)**2) + radius

subdomains = [(1, lambda x: phi_expression(x) <= -tol), 
                (0, lambda x: phi_expression(x) >= -tol)]

cell_tag = tag_subdomains(msh, subdomains) 
dx = Measure("dx", domain=msh, subdomain_data=cell_tag)

2) A partir de uma expressão analitica para phi

3) A partir de uma função phi (só valores discretos conhecidos)

In [ ]:
for N in [10,20,40,80,160,320, 640]:
    msh = create_rectangle(comm=MPI.COMM_WORLD,
                points=((0.0, 0.0), (lx,ly)), n=(N, N),
                cell_type=CellType.triangle, diagonal = _cpp.mesh.DiagonalType.crossed)
    V = fem.FunctionSpace(msh, ("CG", 1))
    phi = fem.Function(V)
    phi.interpolate(phi_expression)
    dx = Measure("dx", domain=msh)
    int_dx = conditional(le(phi, tol), 1, 0)*dx
    ext_dx = conditional(ge(phi, tol), 1, 0)*dx
    vol_teo = pi*radius**2
    vol = fem.assemble_scalar(fem.form(int_dx))

    print('error conditional: ' + str(abs(vol - vol_teo)))

4) No caso de PLSF, em que a expressão anaĺitica pra phi é conhecida: phi(x) =  inner(alpha, )